# Modele analizy danych

### 2024/2025, semestr zimowy
Tomasz Rodak

---

## Optuna

Celem tego arkusza jest zapoznanie się z biblioteką [Optuna](https://optuna.org/). Jest to biblioteka automatyzująca proces optymalizacji funkcji o wartościach rzeczywistych (czyli proces znajdowania minimum lub maksimum lokalnego funkcji). W praktyce Optuna jest typowo stosowana do optymalizacji hiperparametrów modeli uczenia maszynowego.

Podstawowe obiekty definiowane w Optuna to:
- **`Study`** - obiekt reprezentujący proces optymalizacji funkcji celu, składa się na niego wiele prób.
- **`Trial`** - próba, obiekt reprezentujący pojedyńcze obliczenie wartości funkcji celu.

### Przykład
<!-- y = np.exp(np.sin(x**2)) + x**2/5 - x
 -->
Wyznaczymy minimum funkcji lokalne funkcji 
$$
f(x) = \exp\sin x^2 + \frac{x^2}{5} - x
$$
na przedziale $x \in [-5, 5]$.

Wykres:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(-5, 5, 1000)
y = np.exp(np.sin(x**2)) + x**2/5 - x
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(x, y)
ax.grid();

Optymalizacja w Optunie:

In [ ]:
import optuna

a, b = -5, 5

def objective(trial):
    x = trial.suggest_float('x', a, b)
    return np.exp(np.sin(x**2)) + x**2/5 - x

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

Funkcja `objective()` gra rolę funkcji celu, której wartość ma być (w tym przypadku) minimalizowana. Funkcja ta przyjmuje jeden argument `trial`, który jest obiektem reprezentującym pojedyńczą próbę optymalizacyjną. Wiersz

```python
x = trial.suggest_float('x', -5, 5)
```

zmusza Optunę do wskazania zmiennoprzecinkowego kandydata zmiennej `x` z przedziału $[-5, 5]$. Wskazanie to odbywa się w sposób adaptacyjny, tzn. Optuna kieruje się w sugestiach wartościami, które zostały już sprawdzone w poprzednich próbach. Wartość zmiennej `x` jest następnie przekazywana do faktycznej funkcji celu `f(x)`. Ostatecznie `objective()` zwraca wartość funkcji celu dla danego kandydata, która jest zapisywana w obiekcie `trial` i służy do oceny jakości kandydata. Ponieważ naszym celem jest minimalizacja, więc kandydat z mniejszą wartością funkcji celu jest lepszy.

Wywołanie
    
```python
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
```

rozpoczyna proces optymalizacji. Argument `n_trials` określa liczbę prób, które mają być wykonane. Argument `direction` określa kierunek optymalizacji, w tym przypadku minimalizację. Po zakończeniu optymalizacji obiekt `study` zawiera wyniki optymalizacji, w tym najlepsze wartości zmiennych.

Wyniki:


In [ ]:
best_trial = study.best_trial
best_x = best_trial.params['x']
best_y = best_trial.value
print(f'Best x: {best_x:.3f}, Best y: {best_y:.3f}')

Wszystkie próby na wykresie:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(x, y)
for t in study.trials:
    ax.scatter(t.params['x'], t.value, color='red')
ax.scatter(best_x, best_y, color='green')
ax.grid();

### Sampler

Sposób w jaki Optuna wybiera kandydatów do oceny zależy od użytego samplera. Domyślnie Optuna stosuje algorytm optymalizacji bayesowskiej z samplerem `TPESampler` (Tree-structured Parzen Estimator). Możliwe jest również użycie innych samplerów, np. `RandomSampler` czy `GridSampler`. Sampler można ustawić w wywołaniu `create_study()`.

Oto przykład użycia `GridSampler`. W tym przypadku Optuna będzie wybierać kandydatów z siatki przekazanej w argumencie `search_space` samplera. Warto zauważyć, że w przypadku `GridSampler` optymalizacja bayesowska nie ma sensu, ponieważ sprawdzane są wszystkie możliwe wartości. Aby przejść przez wszystkie punkty siatki, `n_trials` musi być ustawione na liczbę punktów w siatce. Ewentualnie można ustawić `n_trials` na `None` - wtedy optymalizacja będzie trwała dopóki nie zostaną sprawdzone wszystkie punkty siatki - lub na wartość mniejszą niż liczba punktów w siatce - wtedy optymalizacja zakończy się po sprawdzeniu `n_trials` punktów, ale może być wznawiana w przyszłości.

In [ ]:
search_space = {
    'x': np.linspace(a, b, 10)
}
study = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space), direction='minimize')
study.optimize(objective, n_trials=None)

In [ ]:
best_trial = study.best_trial
best_x = best_trial.params['x']
best_y = best_trial.value
print(f'Best x: {best_x:.3f}, Best y: {best_y:.3f}')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(x, y)
for t in study.trials:
    ax.scatter(t.params['x'], t.value, color='black')
ax.scatter(best_x, best_y, color='green')
ax.grid();

### Optymalizacja hiperparametrów modelu

Załóżmy, że mamy zbiór obserwacji `X` i `y` wygenerowanych z modelu

\begin{align*}
X &\sim \text{Uniform}(0, 10) \\
Y &\sim \sin(X) + \varepsilon, \quad \varepsilon \sim \mathcal{N}(0, 0.2)
\end{align*}

In [ ]:
N = 100
a, b = 0, 10
X = np.random.uniform(a, b, N)
y = np.sin(X) + np.random.normal(0, 0.2, N)
xx = np.linspace(a, b, 1000)
fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(X, y)
ax.plot(xx, np.sin(xx), color='red', label='$E[Y|X=x]$')
ax.legend();

Naszym celem jest zbudowanie modelu regresji wielomianowej z regularyzacją L2. Model ma wówczas postać

$$
y = \sum_{i=0}^d w_i x^i
$$

gdzie $d$ to stopień wielomianu, a $w_i$ to współczynniki regresji. Regularyzacja L2 polega na dodaniu do funkcji celu kary za duże wartości współczynników:

$$
\text{RSS} + \lambda \sum_{i=0}^d w_i^2
$$

gdzie $\text{RSS}$ to suma kwadratów reszt, a $\lambda$ to współczynnik regularyzacji. 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

model = Pipeline([
    ('poly', PolynomialFeatures(degree=5)),
    ('ridge', Ridge(alpha=1e-3))
])
model.fit(X[:, None], y)

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(X, y)
ax.plot(xx, np.sin(xx), color='red', label='$E[Y|X=x]$')
ax.plot(xx, model.predict(xx[:, None]), color='green', label='Model')
ax.legend();

Hiperparametrami modelu są $d$ i $\lambda$. Naszym celem jest znalezienie takich wartości hiperparametrów, które minimalizują błąd testowy modelu. Przestrzeń hiperparametrów przeszukamy za pomocą Optuny, jak jednak zaimplementować funkcję celu, która powinna zwracać błąd testowy? Błąd testowy obliczamy na tej części danych, które nie były użyta do uczenia modelu. Jedna z prostszych strategii to podzielenie danych na zbiór treningowy i walidacyjny. Część treningową użyjemy do uczenia modelu, a część walidacyjną do obliczenia błędu testowego.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=0)

In [ ]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 0, 100, log=False)
    degree = trial.suggest_int('degree', 1, 20)
    model = Pipeline([
        ('poly', PolynomialFeatures(degree=degree)),
        ('linear', Ridge(alpha=alpha))
    ])
    # test stabilności numerycznej
    XX = model.named_steps['poly'].fit_transform(X_train[:, None])
    Z = XX.T @ XX + alpha * np.eye(XX.shape[1])
    singular_values = np.linalg.svd(Z, compute_uv=False)
    rcond = np.min(singular_values) / np.max(singular_values)
    if rcond < 1e-15:
        return np.inf
    else:
        model.fit(X_train[:, None], y_train)
        y_pred = model.predict(X_val[:, None])
        test_loss = np.mean((y_val - y_pred)**2)
        return test_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

In [ ]:
best_trial = study.best_trial
best_alpha = best_trial.params['alpha']
best_degree = best_trial.params['degree']
best_loss = best_trial.value
print(f'Best alpha: {best_alpha:.3f}, Best degree: {best_degree}, Best loss: {best_loss:.3f}')

In [ ]:
model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('linear', Ridge(alpha=best_alpha))
])
model.fit(X[:, None], y)
yy = model.predict(xx[:, None])

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(X, y)
ax.plot(xx, np.sin(xx), color='red', label='$E[Y|X=x]$')
ax.plot(xx, yy, color='green', label='Model')
ax.legend();

In [ ]:
# visualize the optimization process in dashboard
import optuna.visualization as vis
vis.plot_optimization_history(study)